Temporary Workaround for 

In [ ]:
!pip install -q snscrape
!pip install -q vaderSentiment
!pip install -q transformers

     |████████████████████████████████| 59 kB 1.8 MB/s 
     |████████████████████████████████| 125 kB 10.3 MB/s 
     |████████████████████████████████| 5.8 MB 12.0 MB/s 
     |████████████████████████████████| 7.6 MB 71.4 MB/s 
     |████████████████████████████████| 182 kB 69.9 MB/s 


In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import tweepy
from datetime import date
import textblob
import transformers

In [ ]:
today = date.today()
end_date = today
print(end_date)
search_term = 'IlliniMBB'
from_date = '2022-01-01'

2022-12-04


In [ ]:
os.system(f"snscrape --since {from_date} twitter-search '{search_term} until:{end_date}' > result-tweets.txt")
if os.stat("result-tweets.txt").st_size == 0:
  counter = 0
else:
  df = pd.read_csv('result-tweets.txt', names=['link'])
  counter = df.size

print('Number Of Tweets : '+ str(counter))

Number Of Tweets : 46565


In [ ]:
max_results = 100
extracted_tweets = "snscrape --format '{content!r}'"+ f" --max-results {max_results} --since {from_date} twitter-search '{search_term} until:{end_date}' > extracted-tweets.txt"
os.system(extracted_tweets)
if os.stat("extracted-tweets.txt").st_size == 0:
  print('No Tweets found')
else:
  df = pd.read_csv('extracted-tweets.txt', names=['content'])
  for row in df['content'].iteritems():
    print(row)

(0, "'@IlliniMBB good go fuck yourselves and you and you and you and YOU https://t.co/0F6O9noM4W'")
(1, "'@RizKing4L @IlliniMBB Fr the shit unreal😂 even after Morsell left he STILL beat us too. Something in the water I fear.'")
(2, "'@HoosierSweater @IlliniMBB Big Ten’s good")
(3, "'@CBBonFOX @colehawk23 @IlliniMBB This was right after the announcers acted like Hawkins being on a guard was a mismatch lol'")
(4, "'@GreenStCafeCU @IlliniMBB They’re our daddy we knew this would happen'")
(5, "'@hoosier_tie @IlliniMBB Damn")
(6, "'@HoosierSweater @IlliniMBB HA'")
(7, "'Can we talk about (most) but not all")
(8, "'@IlliniMBB VS @TexasMBB Prediction. https://t.co/6FcGDUQPo5'")
(9, "'@KyleJLeonardd @IlliniMBB @BigTenNetwork @Sn1per_T Yeah you won at home. So now you shoot off your big mouth with a dumb comment. Typical eastern punk no class.'")
(10, "'Watching @IndianaMBB vs @RutgersMBB - Indiana does NOT look like a top 10 team. @IlliniMBB looks like a top 5 team and @TerrapinHoops - even wi

In [ ]:
from transformers import pipeline
sentiment_pipeline = pipeline("sentiment-analysis")
data = df
list = df['content'].astype(str).tolist()


No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


Downloading:   0%|          | 0.00/629 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [ ]:
score_set = sentiment_pipeline(list)
average = 0
count = 0

for x in score_set:
  if (x['label'] == 'NEGATIVE'):
    average = average - (x['score'])
  else:
    average = average + (x['score'])
  count += 1

average = average / count
print("Average sentiment: " + str(average))

Average sentiment: -0.3780542504787445


API Tweepy

In [ ]:
api_key =  "qoHykDTyFDIN0GdlnUHsI5dek"
api_key_secret = "tt3Xe8Za0yuQTjqiXoVWWcmLqfejh41SljZf0uVlvE8JxdPzo5"
access_token = "1587238601766719489-b6AZWjTRlpDi2gFe0spDBeXm8DYt3L"
access_token_secret =   "89MwP7atYbn34psj2VRQh9sCfekLxLSl7pJjUHMbEBbwM"

In [ ]:
authenticator = tweepy.OAuthHandler(api_key, api_key_secret)
authenticator.set_access_token(access_token, access_token_secret)

In [ ]:
api = tweepy.API(authenticator, wait_on_rate_limit=True)

# posts = api.user_timeline(screen_name="BillGates", count = 100, lang="en", tweet_mode="extended")
# print("Show 5 most recent tweets on timeline: /n")
# for tweet in posts[0:5]:
#   print(tweet.full_text + '/n')

In [ ]:
team_dict = {
    "U Illinois": "@IlliniMBB",
    "U Indiana": "@IndianaMBB",
    "U Iowa": "@IowaHoops",
    "U Maryland": "@TerrapinHoops",
    "U Michigan": "@umichbball",
    "U Minnesota": "@GopherMBB",
    "U Nebraska": "@HuskerHoops",
    "U Wisconsin": "@BadgerMBB",
    "Michigan State": "@MSU_Basketball",
    "Penn State": "@PennStateMBB",
    "Ohio State": "@OhioStateHoops",
    "Rutgers": "@RutgersMBB",
    "Purdue": "@BoilerBall",
    "Northwestern": "@NUMensBball",
}

In [ ]:
team = 'Illini'

In [ ]:
search = f'#{team} -filter:retweets'
tweet_cursor = tweepy.Cursor(api.search, q=search, lang='en', tweet_mode='extended').items(500)
tweets = [tweet.full_text for tweet in tweet_cursor]
tweets_df = pd.DataFrame(tweets, columns=['Tweets'])

TweepError: ignored

In [ ]:
for _, row in tweets_df.iterrows():
  row['Tweets'] = re.sub('http\S+', '', row['Tweets'])
  row['Tweets'] = re.sub('#\S+', '', row['Tweets'])
  row['Tweets'] = re.sub('@\S+', '', row['Tweets'])
  row['Tweets'] = re.sub('\\n', '', row['Tweets'])

In [ ]:
tweets_df = df
tweets_df['Polarity'] = tweets_df['Tweets'].map(lambda tweet: textblob.TextBlob(tweet).sentiment.Polarity)
tweets_df['Result'] = tweets_df['Polarity'].map(lambda pol: '+' if pol > 0 else '-')

In [ ]:
positive = tweets_df[tweets_df.Result == '+'].count()['Tweets']
positive = tweets_df[tweets_df.Result == '-'].count()['Tweets']

In [ ]:
plt.bar([0,1], [positive, negative], label=['Positive', 'Negative'], color = ['orange', 'blue'])
plt.show()